# Multi-Instance-Learning for DeliciousMIL

## Imports

In [1]:
import os
import re
import warnings
from itertools import combinations

import numpy as np
import pandas as pd
from keras.utils import pad_sequences
from scipy.spatial.distance import directed_hausdorff 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn_extra.cluster import KMedoids
from sklearn.svm import LinearSVC

from scipy.spatial.distance import directed_hausdorff 
import random

## Data Preprocessing

In [2]:
my_path = os.getcwd()
dataset_dir = my_path + '/DeliciousMIL/Data/'
dataset_dir = dataset_dir.replace('\\', '/')

input_train_data = dataset_dir + 'train-data.dat'
input_test_data = dataset_dir + 'test-data.dat'

input_train_label = dataset_dir + 'train-label.dat'
input_test_label = dataset_dir + 'test-label.dat'

In [3]:
def remove_tags(input_file, output_file):
    with open(input_file, 'r') as file:
        content = file.read()

    pattern = r'<[^>]+>'
    content = re.sub(pattern, '', content)

    with open(output_file, 'w') as file:
        file.write(content)


def replace_multiple_spaces(input_file, output_file):
    with open(output_file, 'w') as file2:
        with open(input_file, 'r') as file:
            for line in file:
                content = line
                content = ' '.join(content.split())

                file2.write(content)
                file2.write("\n")

In [4]:
corpus_X_train = []
with open(input_train_data, 'r') as file:
    for line in file:
        pattern = r'\n'
        line = re.sub(pattern, '', line)
        corpus_X_train.append(line)
        
corpus_X_test = []
with open(input_test_data, 'r') as file:
    for line in file:
        pattern = r'\n'
        line = re.sub(pattern, '', line)
        corpus_X_test.append(line)
        
corpus_y_train = []
with open(input_train_label, 'r') as file:
    for line in file:
        pattern = r'\n'
        line = re.sub(pattern, '', line)
        content = line.split(" ")

        for i in range(0, len(content)):
            content[i] = int(content[i])

        content = np.array(content)

        corpus_y_train.append(content)

corpus_y_test = []
with open(input_test_label, 'r') as file:
    for line in file:
        pattern = r'\n'
        line = re.sub(pattern, '', line)
        content = line.split(" ")

        for i in range(0, len(content)):
            content[i] = int(content[i])

        corpus_y_test.append(content)

We determine the most frequent class among the 20 classes to create a binary classification problem. Following that, a dataframe is generated containing sets of sentences categorized as either belonging or not belonging to the most frequent class. 

In [5]:
def create_bags_of_sentences(documents_corpus: list, labels_corpus: list) -> pd.DataFrame:
    # Count the occurrences of each label class
    labels = np.array(labels_corpus)
    class_counts = np.sum(labels, axis=0)

    # Find the index of the most frequent label class
    most_frequent_class_index = np.argmax(class_counts)

    print("Most frequent class index:", most_frequent_class_index)
    labels = labels[:, most_frequent_class_index]

    # Initialize counters and bag of sentences dictionary
    document_counter = 0
    sentence_count = 0
    bag_of_sentences = {}

    for document, label in zip(documents_corpus, labels):
        # Split document into sentences
        sentences = re.split(r'<\d+>', document)

        for sentence in sentences:
            # Remove leading and trailing whitespaces
            sentence = sentence.strip()

            # If sentence is not empty
            if sentence:
                # Store words as a list of strings
                words = sentence.split(" ")
                # Add a sentence to the bag
                bag_of_sentences[sentence_count] = (document_counter, words, label)
                sentence_count += 1

        document_counter += 1

    # Create dataframe of the bag
    df = pd.DataFrame.from_dict(bag_of_sentences, orient='index',
                                columns=['Bag', 'Sentence', 'Class'])

    return df


In [6]:
train_df = create_bags_of_sentences(corpus_X_train, corpus_y_train)
test_df = create_bags_of_sentences(corpus_X_test, corpus_y_train)
print(train_df.head())

Most frequent class index: 2
Most frequent class index: 2
   Bag                                           Sentence  Class
0    0    [6705, 5997, 8310, 3606, 674, 8058, 5044, 4836]      1
1    0                           [4312, 5154, 8310, 4225]      1
2    1                            [1827, 1037, 8482, 483]      1
3    1  [3567, 6172, 6172, 2892, 1362, 787, 399, 777, ...      1
4    1      [318, 769, 4621, 3199, 1480, 6213, 971, 6890]      1


### Transforming the dataframe into a bag of sentences per document.

In [7]:
def create_bag_per_document(df: pd.DataFrame):

    ids, X, y = np.array(df['Bag']), np.array(df['Sentence']), np.array(df['Class'])
    un_id = np.unique(ids)
    data = []
    labels = []

    for i in range(un_id.shape[0]):
        bag = X[ids == un_id[i]]
        data.append(bag)
        label = y[ids == un_id[i]]
        labels.append(label)

    data = np.array(data, dtype=object)
    labels = np.array(labels, dtype=object)
    labels = np.array([labels[i][0] for i in range(labels.shape[0])])
    
    # Pad sentences for 40 words per sentence max.
    for i, sentence in enumerate(data):
        data[i] = pad_sequences(sentence, maxlen=40)

    return data, labels

# Get the bags and the labels.
train_bag, y_train = create_bag_per_document(train_df)
test_bag, y_test = create_bag_per_document(test_df)

### Clustering

We conduct clustering using Hausdorff distance and K-Medoids. Initially, distances for all our data are computed to generate the distance matrix. Subsequently, employing the K-Medoids method, we choose to cluster our data into 3 clusters based on experimentation, which yielded the best silhouette score. Finally, we transform the data and execute the clustering process.

In [8]:
def calculate_hausdorff(x, y):
    return max(directed_hausdorff(x, y)[0], directed_hausdorff(y, x)[0])

# Initialize distance matrix.
n_data = train_bag.shape[0]
distance_matrix = np.zeros((n_data, n_data))

# Calculate symmetric haussdorff distances.
for (i, x), (j, y) in combinations(enumerate(train_bag), 2):
    distance_matrix[i, j] = calculate_hausdorff(x, y)
    distance_matrix[j, i] = distance_matrix[i, j]


KeyboardInterrupt



In [ ]:
from multiprocessing import Pool

def calculate_hausdorff(args):
    i, j, x, y = args
    return i, j, max(directed_hausdorff(x, y)[0], directed_hausdorff(y, x)[0])

def calculate_distance_matrix(args):
    i, x, train_bag = args
    distances = [calculate_hausdorff((i, j, x, y)) for j, y in enumerate(train_bag)]
    return distances

# Initialize distance matrix.
n_data = train_bag.shape[0]
distance_matrix = np.zeros((n_data, n_data))

# Use multiprocessing for parallel processing.
with Pool() as pool:
    args_list = [(i, x, train_bag) for i, x in enumerate(train_bag)]
    distance_matrix_list = pool.map(calculate_distance_matrix, args_list)

# Fill in the distance matrix.
for i, distances in enumerate(distance_matrix_list):
    for j, distance in enumerate(distances):
        distance_matrix[i, j] = distance[2]
        distance_matrix[j, i] = distance[2]

In [ ]:
init_medoids_indices = random.sample(range(distance_matrix.shape[0]), 3)
init_medoids = train_bag[init_medoids_indices]

def k_medoids(distance_matrix, k, max_iter=100):
    n_data = distance_matrix.shape[0]
    medoid_indices = np.array(init_medoids_indices)
    labels = np.zeros(n_data, dtype=int)
    
    for _ in range(max_iter):
        distances = distance_matrix[:, medoid_indices]
        labels = np.argmin(distances, axis=1)
        
        new_medoid_indices = np.empty(k, dtype=int)
        for cluster_label in range(k):
            cluster_indices = np.where(labels == cluster_label)[0]
            cluster_distances = distances[cluster_indices, cluster_label]
            new_medoid_indices[cluster_label] = cluster_indices[np.argmin(cluster_distances)]
        
        if np.array_equal(medoid_indices, new_medoid_indices):
            break
        
        medoid_indices = new_medoid_indices
    
    return medoid_indices, labels

final_medoids_indices, cluster_labels = k_medoids(distance_matrix, 3)

final_medoids = train_bag[final_medoids_indices]

In [ ]:
def generate_features(data_bag, medoids) -> np.ndarray:

    data_transformed = np.empty((len(data_bag), len(medoids)))

    # Generate features, using the distances from the medoids.
    for i, x in enumerate(data_bag):
        for j, medoid in enumerate(medoids):
            data_transformed[i, j] = calculate_hausdorff(x, medoid)

    return data_transformed

# Transform data.
X_train_transformed = generate_features(train_bag, final_medoids)
X_test_transformed = generate_features(test_bag, final_medoids)

### Transformed dataframe.

In [ ]:
print(f'New X train data shape: {X_train_transformed.shape}')
print(f'New X test  data shape: {X_test_transformed.shape}')

In [ ]:
models = []
tree = DecisionTreeClassifier(criterion='entropy', max_depth=2, max_features=2, random_state=0)
models.append([tree, "Decision Tree"])
svm = LinearSVC(C=0.01)
models.append([svm, "Linear SVM"])

for classifier, name in models:
    print(name)
    clf = classifier.fit(X_train_transformed, y_train)
    y_pred = clf.predict(X_test_transformed)
    print(f'Acuracy: {accuracy_score(y_test, y_pred)}')
    print(f'Precision: {precision_score(y_test, y_pred, average="macro")}')
    print(f'Recall: {recall_score(y_test, y_pred, average="macro")}')
    print(f'F1: {f1_score(y_test, y_pred, average="macro")}')

### Results

After conducting various experiments with different algorithms, it was observed that the tree-based models exhibited the best performance, albeit still falling short of the desired outcome. The likelihood of substantial information loss is high when considering each sentence as an individual instance in our data. We hypothesize that better results could be achieved by treating each word as an instance and utilizing generalized bags of words.